In [8]:
import pandas as pd
import numpy as np
import time

from methods.direct_methods import (
    naive_gauss,
    gauss_partial_pivoting,
    gauss_scaled_pivoting,
    gauss_complete_pivoting,
    LU_solve,
)

from methods.iterative_methods import (
    jacobi,
    gauss_seidel,
    relaxing,
)

from utils.matrix import generate_hilbert_matrix

# Resolução de sistemas de equações lineares

## 1. Exercício

### Condicionamento do sistema

In [9]:
determinants = []
cond_numbers = []

ns = [5, 9, 15]

for n in ns:
    A, B = generate_hilbert_matrix(n=n)
    determinants.append(np.linalg.det(A))
    cond_numbers.append(np.linalg.cond(A))

for n, det in zip(ns, determinants):
    print(f"Determinante da matriz de Hilbert de ordem {n}: {det:.4e}")

print("\n")

for n, cond in zip(ns, cond_numbers):
    print(f"Número de condição da matriz de Hilbert de ordem {n}: {cond:.4e}")


Determinante da matriz de Hilbert de ordem 5: 3.7493e-12
Determinante da matriz de Hilbert de ordem 9: 9.7203e-43
Determinante da matriz de Hilbert de ordem 15: -1.2321e-120


Número de condição da matriz de Hilbert de ordem 5: 4.7661e+05
Número de condição da matriz de Hilbert de ordem 9: 4.9315e+11
Número de condição da matriz de Hilbert de ordem 15: 2.4960e+17


Ao analisar os resultados obtidos, observou-se que a matriz de Hilbert está mau condicionada para todos os tamanhos testados. O condicionamento da matriz se tornou pior com dimensões maiores.

Para o teste do determinante, quanto maior a dimensão da matriz, mais o determinante se aproximou de zero, o que indica uma piora no condicionamento.

No teste do número de condição da matriz, o número de condicionamento aumentou para dimensões maiores. Nesse teste, um valor unitário indica bom condicionamento e valores grandes ou distante do valor unitário indicam mau condicionamento.

### Resolução do sistema

In [10]:
A_5, B_5 = generate_hilbert_matrix(n=5)
A_5_augmented = np.hstack((A_5, B_5.reshape(-1, 1)))

A_9, B_9 = generate_hilbert_matrix(n=9)
A_9_augmented = np.hstack((A_9, B_9.reshape(-1, 1)))

A_15, B_15 = generate_hilbert_matrix(n=15)
A_15_augmented = np.hstack((A_15, B_15.reshape(-1, 1)))

result_dict = {}
A_augmented_list = [A_5_augmented, A_9_augmented, A_15_augmented]
A_list = [A_5, A_9, A_15]

#### Métodos diretos

In [11]:
for n, A_i, A_augmented in zip(ns, A_list, A_augmented_list):
    X_true = np.ones_like(A_i[:, 0])

    start_time = time.perf_counter()
    X_naive = naive_gauss(A_augmented)
    gauss_naive_execution_time = time.perf_counter() - start_time
    abs_error_naive = np.linalg.norm(X_naive - X_true)
    rel_error_naive = abs_error_naive / np.linalg.norm(X_true)

    start_time = time.perf_counter()
    X_partial = gauss_partial_pivoting(A_augmented)
    gauss_partial_execution_time = time.perf_counter() - start_time
    abs_error_partial = np.linalg.norm(X_partial - X_true)
    rel_error_partial = abs_error_partial / np.linalg.norm(X_true)

    start_time = time.perf_counter()
    X_scaled = gauss_scaled_pivoting(A_augmented)
    gauss_scaled_execution_time = time.perf_counter() - start_time
    abs_error_scaled = np.linalg.norm(X_scaled - X_true)
    rel_error_scaled = abs_error_scaled / np.linalg.norm(X_true)

    start_time = time.perf_counter()
    X_complete = gauss_complete_pivoting(A_augmented)
    gauss_complete_execution_time = time.perf_counter() - start_time
    abs_error_complete = np.linalg.norm(X_complete - X_true)
    rel_error_complete = abs_error_complete / np.linalg.norm(X_true)

    result_dict[n] = {
        "gauss_naive": {
            "result": X_naive,
            "execution_time": gauss_naive_execution_time,
            "abs_error": abs_error_naive,
            "rel_error": rel_error_naive,
        },
        "gauss_partial": {
            "result": X_partial,
            "execution_time": gauss_partial_execution_time,
            "abs_error": abs_error_partial,
            "rel_error": rel_error_partial,
        },
        "gauss_scaled": {
            "result": X_scaled,
            "execution_time": gauss_scaled_execution_time,
            "abs_error": abs_error_scaled,
            "rel_error": rel_error_scaled,
        },
        "gauss_complete": {
            "result": X_complete,
            "execution_time": gauss_complete_execution_time,
            "abs_error": abs_error_complete,
            "rel_error": rel_error_complete,
        },
    }

##### Tempo de execução e precisão

In [12]:
df = pd.DataFrame.from_dict(
    {
        "n": [],
        "method": [],
        "execution_time": [],
        "abs_error": [],
        "rel_error": [],
    }
)
for n, results in result_dict.items():
    for method, result in results.items():
        df = pd.concat(
            [
                df,
                pd.DataFrame(
                    {
                        "n": [n],
                        "method": [method],
                        "execution_time": [result["execution_time"]],
                        "abs_error": [result["abs_error"]],
                        "rel_error": [result["rel_error"]],
                    }
                ),
            ],
            ignore_index=True,
        )
df["execution_time"] = df["execution_time"].apply(lambda x: f"{x:.7e} s")
df["abs_error"] = df["abs_error"].apply(lambda x: f"{x:.7e}")
df["rel_error"] = df["rel_error"].apply(lambda x: f"{x:.7e}")
df

,n,method,execution_time,abs_error,rel_error
0,5.0,gauss_naive,5.1799994e-05 s,3.4726990e-12,1.5530382e-12
1,5.0,gauss_partial,5.1800002e-05 s,3.4704562e-12,1.5520352e-12
2,5.0,gauss_scaled,1.0820000e-04 s,3.4704562e-12,1.5520352e-12
3,5.0,gauss_complete,6.7500005e-05 s,3.4706916e-12,1.5521405e-12
4,9.0,gauss_naive,1.5210000e-04 s,2.5826034e-05,8.6086779e-06
5,9.0,gauss_partial,1.9810000e-04 s,2.5825986e-05,8.6086621e-06
6,9.0,gauss_scaled,2.5410000e-04 s,2.5825986e-05,8.6086621e-06
7,9.0,gauss_complete,3.0830000e-04 s,2.5825986e-05,8.6086621e-06
8,15.0,gauss_naive,1.3839000e-03 s,1.3060006e+01,3.3720791e+00
9,15.0,gauss_partial,1.4120000e-03 s,1.2991236e+01,3.3543228e+00


##### Valores de X de cada método

In [13]:
max_n = max(ns)
df_X = pd.DataFrame(columns=["n", "method"] + [f"x{i+1}" for i in range(max_n)])

for n, results in result_dict.items():
    for method in ["gauss_naive", "gauss_partial", "gauss_scaled", "gauss_complete"]:
        X = results[method]["result"]
        # Preenche com NaN se X for menor que max_n
        row = [n, method] + list(X) + [np.nan] * (max_n - len(X))
        row = row[:2 + max_n]  # Garante o tamanho correto
        df_X.loc[len(df_X)] = row

df_X

,n,method,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15
0,5,gauss_naive,1.0,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,gauss_partial,1.0,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,gauss_scaled,1.0,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,gauss_complete,1.0,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,gauss_naive,1.0,1.000000,1.000000,1.000002,0.999993,1.000015,0.999983,1.000010,0.999997,NaN,NaN,NaN,NaN,NaN,NaN
5,9,gauss_partial,1.0,1.000000,1.000000,1.000002,0.999993,1.000015,0.999983,1.000010,0.999997,NaN,NaN,NaN,NaN,NaN,NaN
6,9,gauss_scaled,1.0,1.000000,1.000000,1.000002,0.999993,1.000015,0.999983,1.000010,0.999997,NaN,NaN,NaN,NaN,NaN,NaN
7,9,gauss_complete,1.0,1.000000,1.000000,1.000002,0.999993,1.000015,0.999983,1.000010,0.999997,NaN,NaN,NaN,NaN,NaN,NaN
8,15,gauss_naive,1.0,1.000002,0.999995,0.998640,1.025668,0.781933,2.066841,-2.279037,7.532393,-7.355048,7.380667,-1.129041,0.425749,1.733284,0.817952
9,15,gauss_partial,1.0,1.000002,0.999996,0.998634,1.025663,0.782322,2.063961,-2.267832,7.505387,-7.312708,7.337388,-1.101271,0.415689,1.734771,0.817999


#### Métodos iterativos

In [14]:
for n, A_i, A_augmented in zip(ns, A_list, A_augmented_list):
    X_true = np.ones_like(A_i[:, 0])

    start_time = time.perf_counter()
    X_jacobi = jacobi(A_augmented, X_true)
    jacobi_execution_time = time.perf_counter() - start_time
    abs_error_jacobi = np.linalg.norm(X_jacobi - X_true)
    rel_error_jacobi = abs_error_jacobi / np.linalg.norm(X_true)

    start_time = time.perf_counter()
    X_gauss_seidel = gauss_seidel(A_i, B_i, X_true)
    gauss_seidel_execution_time = time.perf_counter() - start_time
    abs_error_gauss_seidel = np.linalg.norm(X_gauss_seidel - X_true)
    rel_error_gauss_seidel = abs_error_gauss_seidel / np.linalg.norm(X_true)

    start_time = time.perf_counter()
    X_relaxing = relaxing(A_i, B_i, X_true)
    relaxing_execution_time = time.perf_counter() - start_time
    abs_error_relaxing = np.linalg.norm(X_relaxing - X_true)
    rel_error_relaxing = abs_error_relaxing / np.linalg.norm(X_true)

    result_dict[n].update(
        {
            "jacobi": {
                "result": X_jacobi,
                "execution_time": jacobi_execution_time,
                "abs_error": abs_error_jacobi,
                "rel_error": rel_error_jacobi,
            },
            "gauss_seidel": {
                "result": X_gauss_seidel,
                "execution_time": gauss_seidel_execution_time,
                "abs_error": abs_error_gauss_seidel,
                "rel_error": rel_error_gauss_seidel,
            },
            "relaxing": {
                "result": X_relaxing,
                "execution_time": relaxing_execution_time,
                "abs_error": abs_error_relaxing,
                "rel_error": rel_error_relaxing,
            },
        }
    )

TypeError: jacobi() missing 1 required positional argument: 'max_iter'